# Generate Floodplain Sampling Sets

**Timm Nawrocki**  
Alaska Center for Conservation Science  
2019-04-06

In [1]:
# -*- coding: utf-8 -*-
# ---------------------------------------------------------------------------
# Generate Floodplain Sampling Sets
# Author: Timm Nawrocki
# Created on: 2019-04-05
# Usage: Must be executed as a Jupyter Notebook in an ArcGIS Pro Python 3 installation.
# Description: "Generate Floodplain Sampling Sets" generates the stratified-random sampling points for the Nuyakuk-Nushagak rivers sampling area and the Aleknagik - Wood River sampling area. Points are stratified by estimated floodplain corridor, buffered floodplain corridor, and non-floodplain for the subwatersheds of each sampling area.
# ---------------------------------------------------------------------------

## 1. Initialize Environment

In [2]:
# Import packages
import arcpy
from arcpy.sa import *
import datetime
import os
import time

# Set overwrite option
arcpy.env.overwriteOutput = True

# Set root directory
drive = 'E:/'
root_directory = os.path.join(drive, 'ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS')

# Set arcpy working environment
arcpy.env.workspace = os.path.join(root_directory, 'BristolBay_Vegetation.gdb')

# Define script paths
script_directory = os.path.join(drive, 'ACCS_Work/Repositories/southwest-alaska-moose/modules')
arcpy_geoprocessing_script = os.path.join(script_directory, 'arcpy_geoprocessing.py')

# Define input datasets
floodplains_initial = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_Floodplains_Initial')
nuyakuk_subwatersheds = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_NuyakukNushagak_Subwatersheds')
aleknagik_subwatersheds = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_AleknagikWood_Subwatersheds')
area_of_interest = os.path.join(root_directory, 'Data_Input/area_of_interest/area_of_interest.tif')

# Define intermediate stratification datasets
aleknagik_floodplain = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_FloodplainUnit_Aleknagik')
aleknagik_buffer = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_BufferUnit_Aleknagik')
aleknagik_upland = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_UplandUnit_Aleknagik')
nuyakuk_floodplain = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_FloodplainUnit_Nuyakuk')
nuyakuk_buffer = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_BufferUnit_Nuyakuk')
nuyakuk_upland = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_UplandUnit_Nuyakuk')

# Define output datasets
aleknagik_floodplain_sample = os.path.join(root_directory, 'Data_Input/floodplain_points/FloodplainSample_Aleknagik.shp')
nuyakuk_floodplain_sample = os.path.join(root_directory, 'Data_Input/floodplain_points/FloodplainSample_Nuyakuk.shp')

In [3]:
# Import and execute arcpy_geoprocessing.py
try:
    exec(open(arcpy_geoprocessing_script).read())
except:
    print("Error loading arcpy_geoprocessing.py; ensure that script directory is correct:")
    print(script_directory)
    quit()

## 2. Generate Stratification Units

In [4]:
# Define a function to create stratification units from the floodplain estimate and sampling watersheds
def floodplain_stratification(**kwargs):
    """
    Description: generates floodplain, buffer, and upland stratification units in a sampling subwatershed set
    Inputs: 'input_array' -- an array containing an area of interest raster, a floodplain estimate feature class, and a sampling subwatersheds feature class
            'output_array' -- the output should be an array of a a floodplain feature class, a buffer feature class, and an upland feature class
    """
    # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    area_of_interest = kwargs['input_array'][0]
    floodplains = kwargs['input_array'][1]
    subwatersheds = kwargs['input_array'][2]
    floodplain_unit = kwargs['output_array'][0]
    buffer_unit = kwargs['output_array'][1]
    upland_unit = kwargs['output_array'][2]
    # Define intermediate datasets
    aoi_polygon = os.path.join(arcpy.env.workspace, 'aoi_polygon')
    subwatersheds_clip = os.path.join(arcpy.env.workspace, 'subwatersheds_clip')
    floodplains_clip = os.path.join(arcpy.env.workspace, 'floodplains_clip')
    buffer = os.path.join(arcpy.env.workspace, 'buffer')
    subwatersheds_dissolve = os.path.join(arcpy.env.workspace, 'subwatersheds_dissolve')
    # Convert area of interest to polygon
    print('Converting area of interest to polygon...')
    arcpy.RasterToPolygon_conversion(area_of_interest, aoi_polygon, 'NO_SIMPLIFY', 'VALUE', 'MULTIPLE_OUTER_PART')
    # Clip subwatersheds to area of interest
    print('Modifying subwatersheds to area of interest...')
    arcpy.Clip_analysis(subwatersheds, aoi_polygon, subwatersheds_clip)
    # Clip floodplains to subwatersheds and dissolve
    print('Creating floodplain unit...')
    arcpy.Clip_analysis(floodplains, subwatersheds_clip, floodplains_clip)
    arcpy.Dissolve_management(floodplains_clip, floodplain_unit, '', '', 'MULTI_PART', 'DISSOLVE_LINES')
    # Buffer the floodplains by 500 meters and erase the floodplains
    print('Creating buffer unit...')
    arcpy.Buffer_analysis(floodplain_unit, buffer, '500 Meters', 'FULL', 'ROUND', 'ALL', '', 'PLANAR')
    arcpy.Erase_analysis(buffer, floodplain_unit, buffer_unit)
    # Dissolve the subwatersheds and erase the buffered floodplains
    print('Creating upland unit...')
    arcpy.Dissolve_management(subwatersheds_clip, subwatersheds_dissolve, '', '', 'MULTI_PART', 'DISSOLVE_LINES')
    arcpy.Erase_analysis(subwatersheds_dissolve, buffer, upland_unit)
    # Delete intermediate datasets
    arcpy.Delete_management(aoi_polygon)
    arcpy.Delete_management(subwatersheds_clip)
    arcpy.Delete_management(floodplains_clip)
    arcpy.Delete_management(buffer)
    arcpy.Delete_management(subwatersheds_dissolve)
    # End timing function execution and calculate elapsed time
    end = time.time()
    elapsed = int(end - start)
    success_time = datetime.datetime.now()
    # Report process success
    print('Successfully created floodplain sampling stratification units...')
    print('----------')
    out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                datetime.timedelta(seconds=elapsed))
    return out_process

In [5]:
# Define key word values for Nuyakuk and Nushagak Rivers
nuyakuk_stratification_inputs = [area_of_interest, floodplains_initial, nuyakuk_subwatersheds]
nuyakuk_stratification_outputs = [nuyakuk_floodplain, nuyakuk_buffer, nuyakuk_upland]

# Create key word arguments
nuyakuk_stratification_kwargs = {'input_array' : nuyakuk_stratification_inputs,
                                 'output_array' : nuyakuk_stratification_outputs
                                }

# Create the Nuyakuk and Nushagak stratification units
arcpy_geoprocessing(floodplain_stratification, **nuyakuk_stratification_kwargs)

Converting area of interest to polygon...
Modifying subwatersheds to area of interest...
Creating floodplain unit...
Creating buffer unit...
Creating upland unit...
Successfully created floodplain sampling stratification units...
----------
Succeeded at 2019-04-06 16:43 (Elapsed time: 0:17:21)


In [6]:
# Define key word values for Aleknagik and Wood River
aleknagik_stratification_inputs = [area_of_interest, floodplains_initial, aleknagik_subwatersheds]
aleknagik_stratification_outputs = [aleknagik_floodplain, aleknagik_buffer, aleknagik_upland]

# Create key word arguments
aleknagik_stratification_kwargs = {'input_array' : aleknagik_stratification_inputs,
                                   'output_array' : aleknagik_stratification_outputs
                                  }

# Create the Aleknagik and Wood River stratification units
arcpy_geoprocessing(floodplain_stratification, **aleknagik_stratification_kwargs)

Converting area of interest to polygon...
Modifying subwatersheds to area of interest...
Creating floodplain unit...
Creating buffer unit...
Creating upland unit...
Successfully created floodplain sampling stratification units...
----------
Succeeded at 2019-04-06 17:00 (Elapsed time: 0:17:00)


## 3. Create random samples

In [13]:
# Define a function to create a random sample from stratification units
def random_stratified(**kwargs):
    """
    Description: generates floodplain, buffer, and upland stratification units in a sampling subwatershed set
    Inputs: 'input_array' -- an array containing any number of stratification unit feature classes
            'output_array' -- the output should be an array of a single output point feature class
    """
    # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    input_array = kwargs['input_array']
    output_points = kwargs['output_array'][0]
    # Generate 100 random points per stratification unit
    count = 1
    number = len(input_array)
    points_array = []
    for unit in input_array:
        print('Generating random sample set {0} of {1}...'.format(count, number))
        unit_points = unit + '_Points'
        points_array.append(unit_points)
        output_path, output_name = os.path.split(unit_points)
        arcpy.CreateRandomPoints_management(output_path, output_name, unit, '', 100, '200 Meters', 'POINT')
        count += 1
    # Merge points into output
    print('Merging random sample sets into output...')
    arcpy.Merge_management(points_array, output_points)
    arcpy.AddField_management(output_points, 'FLOOD', 'SHORT', '', '', '', '', 'NULLABLE', 'NON_REQUIRED', '')
    # Delete intermediate datasets
    for unit_points in points_array:
        arcpy.Delete_management(unit_points)
    # End timing function execution and calculate elapsed time
    end = time.time()
    elapsed = int(end - start)
    success_time = datetime.datetime.now()
    # Report process success
    print('Successfully created stratified-random floodplain sampling points...')
    print('----------')
    out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                datetime.timedelta(seconds=elapsed))
    return out_process

In [14]:
# Define key word values for Nuyakuk and Nushagak Rivers
nuyakuk_sample_inputs = [nuyakuk_floodplain, nuyakuk_buffer, nuyakuk_upland]
nuyakuk_sample_outputs = [nuyakuk_floodplain_sample]

# Create key word arguments
nuyakuk_sample_kwargs = {'input_array' : nuyakuk_sample_inputs,
                         'output_array' : nuyakuk_sample_outputs
                        }

# Create the Nuyakuk and Nushagak floodplain sampling points
arcpy_geoprocessing(random_stratified, **nuyakuk_sample_kwargs)

E:/ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS\Data_Input/floodplain_points/FloodplainSample_Nuyakuk.shp already exists and will be overwritten.
Generating random sample set 1 of 3...
Generating random sample set 2 of 3...
Generating random sample set 3 of 3...
Merging random sample sets into output...
Successfully created stratified-random floodplain sampling points...
----------
Succeeded at 2019-04-06 17:09 (Elapsed time: 0:00:14)


In [15]:
# Define key word values for Aleknagik and Wood River
aleknagik_sample_inputs = [aleknagik_floodplain, aleknagik_buffer, aleknagik_upland]
aleknagik_sample_outputs = [aleknagik_floodplain_sample]

# Create key word arguments
aleknagik_sample_kwargs = {'input_array' : aleknagik_sample_inputs,
                           'output_array' : aleknagik_sample_outputs
                          }

# Create the Aleknagik and Wood River floodplain sampling points
arcpy_geoprocessing(random_stratified, **aleknagik_sample_kwargs)

E:/ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS\Data_Input/floodplain_points/FloodplainSample_Aleknagik.shp already exists and will be overwritten.
Generating random sample set 1 of 3...
Generating random sample set 2 of 3...
Generating random sample set 3 of 3...
Merging random sample sets into output...
Successfully created stratified-random floodplain sampling points...
----------
Succeeded at 2019-04-06 17:09 (Elapsed time: 0:00:12)
